# US Freight Industry

This is the final project for the Data Science course by Cambridge ICU, delivered by Flatiron School.

In [1]:
# Import libraries required
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
regions_df = pd.read_csv('FAF5.3.csv',
                         dtype = {'dms_orig': str, 'dms_dest': str})

In [10]:
regions_df.columns

Index(['fr_orig', 'dms_orig', 'dms_dest', 'fr_dest', 'fr_inmode', 'dms_mode',
       'fr_outmode', 'sctg2', 'trade_type', 'dist_band', 'tons_2017',
       'tons_2018', 'tons_2019', 'tons_2020', 'tons_2022', 'tons_2023',
       'tons_2025', 'tons_2030', 'tons_2035', 'tons_2040', 'tons_2045',
       'tons_2050', 'value_2017', 'value_2018', 'value_2019', 'value_2020',
       'value_2022', 'value_2023', 'value_2025', 'value_2030', 'value_2035',
       'value_2040', 'value_2045', 'value_2050', 'current_value_2018',
       'current_value_2019', 'tmiles_2017', 'tmiles_2018', 'tmiles_2019',
       'tmiles_2020', 'tmiles_2022', 'tmiles_2023', 'tmiles_2025',
       'tmiles_2030', 'tmiles_2035', 'tmiles_2040', 'tmiles_2045',
       'tmiles_2050'],
      dtype='object')

In [12]:
dom_origin_df = regions_df.loc[pd.isnull(regions_df['fr_orig'])]

,fr_orig,dms_orig,dms_dest,fr_dest,fr_inmode,dms_mode,fr_outmode,sctg2,trade_type,dist_band,...,tmiles_2019,tmiles_2020,tmiles_2022,tmiles_2023,tmiles_2025,tmiles_2030,tmiles_2035,tmiles_2040,tmiles_2045,tmiles_2050
0,NaN,011,011,NaN,NaN,1,NaN,1,1,1,...,3.351765,3.398871,3.614785,3.718716,4.364371,5.327137,6.342501,7.763605,9.571647,11.343818
1,NaN,011,019,NaN,NaN,1,NaN,1,1,2,...,50.646235,51.702439,54.030117,55.009694,62.052489,72.493059,82.113979,96.869428,116.602924,134.683927
2,NaN,011,129,NaN,NaN,1,NaN,1,1,3,...,0.468714,0.476138,0.483170,0.481371,0.486727,0.533539,0.564126,0.638719,0.750361,0.844720
3,NaN,011,131,NaN,NaN,1,NaN,1,1,2,...,2.267241,2.372436,2.359789,2.352784,2.549854,3.018065,3.155784,3.394250,3.850035,4.178881
4,NaN,011,139,NaN,NaN,1,NaN,1,1,2,...,1.327792,1.421785,1.412938,1.412791,1.554450,1.863748,1.969298,2.141527,2.456208,2.683008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2398858,NaN,559,061,808.0,NaN,1,3.0,43,3,8,...,0.000000,0.244204,0.267250,0.275731,0.294570,0.345673,0.396910,0.458420,0.532266,0.620986
2398859,NaN,559,171,808.0,NaN,1,4.0,43,3,2,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2398860,NaN,559,279,808.0,NaN,7,7.0,43,3,3,...,0.000000,1.722202,1.783264,1.827262,1.936252,2.203291,2.484983,2.819591,3.217100,3.689749
2398861,NaN,559,512,808.0,NaN,1,3.0,43,3,6,...,0.000000,0.000426,0.000512,0.000535,0.000579,0.000698,0.000819,0.000964,0.001140,0.001352
